In [ ]:
import random

import numpy as np

import plotly.express as px

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

from instance_mongodb import instance_mongodb_sei

In [51]:
def get_quantities(cursor):
    """Get quantities that are needed for plotting."""
    for document in cursor:

        alpha_eigenvalues = document["calcs_reversed"][0]["alpha_eigenvalues"]
        final_energy = document["output"]["final_energy"]
        alpha_coeff_matrix = document["calcs_reversed"][0]["alpha_coeff_matrix"]

        alpha_eigenvalues = np.sort(alpha_eigenvalues)
        positive_alpha_eigenvalues = alpha_eigenvalues[alpha_eigenvalues > 0]
        negative_alpha_eigenvalues = alpha_eigenvalues[alpha_eigenvalues < 0]

        homo = negative_alpha_eigenvalues[-1] 
        lumo = positive_alpha_eigenvalues[0]

        yield {
            "final_energy": final_energy,
            "alpha_eigenvalues": alpha_eigenvalues,
            "alpha_coeff_matrix": alpha_coeff_matrix,
        } 

db = instance_mongodb_sei(project="mlts")

collections_data = db.minimal_basis

# Create a new collection to store the data in
collections_data_new = db.minimal_basis_interpolated_sn2
groupname = "sn2_interpolated_from_transition_states"

# Find all unique reaction labels
reaction_labels = collections_data.distinct("tags.label")
# Randomlise the reaction labels
random.shuffle(reaction_labels)

df = pd.DataFrame()

for reaction_label in reaction_labels:

    cursor = (
        collections_data.find(
            {
                "tags.label": reaction_label,
                "tags.group": groupname,
            }
        )
        .sort("tags.scaling", 1)
    )
    data_to_plot = list(get_quantities(cursor))

    selected_eigenval = data_to_plot[0]["alpha_eigenvalues"]
    selected_eigenval = selected_eigenval[selected_eigenval < 0]
    selected_eigenval = np.sort(selected_eigenval)
    selected_eigenval = selected_eigenval[-1]
    selected_eigenval_index = np.where(
        data_to_plot[0]["alpha_eigenvalues"] == selected_eigenval
    )[0][0]
    coeff_matrices = []
    for data in data_to_plot:
        selected_coeff_matrix = data["alpha_coeff_matrix"]
        selected_coeff_matrix = np.array(selected_coeff_matrix)
        selected_coeff_matrix = selected_coeff_matrix[:, selected_eigenval_index]
        coeff_matrices.append(selected_coeff_matrix)
    coeff_matrices = np.array(coeff_matrices)
    coeff_matrices = coeff_matrices[:, np.newaxis, :]

    fig = px.imshow(
        np.abs(coeff_matrices),
        labels=dict(
            x="Basis Index", 
        ),
        title=f"Reaction {reaction_label}",
        animation_frame=0,
    )
    # Remove y-ticks
    fig.update_yaxes(showticklabels=False)
    fig.show()

    break
